In [1]:
import os

In [2]:
os.listdir('./severstal-steel-defect-detection/')

['sample_submission.csv',
 'test_images',
 'test_images.zip',
 'train.csv',
 'train_images',
 'train_images.zip']

In [3]:
from __future__ import print_function, division, absolute_import

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from collections import OrderedDict

# import data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# import image utils
import PIL
from PIL import Image, ImageOps, ImageEnhance

# import image processing
import scipy.ndimage as ndi
import scipy

# import image utilities
from skimage.morphology import binary_opening, disk, label, binary_closing

# import image augmentation
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, RandomBrightnessContrast, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, Flip, OneOf, Compose, PadIfNeeded, RandomContrast, RandomGamma, RandomBrightness, ElasticTransform,
    NoOp, RandomSizedCrop, RGBShift, VerticalFlip, RandomRotate90, Normalize, Resize
)
from albumentations.pytorch.transforms import ToTensor

# Import PyTorch
import torch
from torch import nn
from torch import optim
from torch.optim import Optimizer
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.functional as TF
from torch.utils.data.sampler import SubsetRandomSampler, Sampler
from torch.autograd import Variable
import torch.utils.model_zoo as model_zoo
from torch.nn import init
from torch.optim import lr_scheduler

import math
import os

import time

from tqdm import tqdm_notebook

import cv2
from skimage.morphology import binary_opening, disk, label

import segmentation_models_pytorch as smp

In [4]:
TRAIN_PATH = './severstal-steel-defect-detection/train_images/'
TEST_PATH = './severstal-steel-defect-detection/test_images/'

train = pd.read_csv('./severstal-steel-defect-detection/train.csv')


train_transforms = [
    OneOf([
        ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1,
                         rotate_limit=15,
                         border_mode=cv2.BORDER_CONSTANT),
        OpticalDistortion(distort_limit=0.11, shift_limit=0.15,
                          border_mode=cv2.BORDER_CONSTANT),
        ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
        NoOp()]),
    RandomSizedCrop(min_max_height=(100, 256),
                    height=256,
                    width=1600, p=0.3),
    HorizontalFlip(p=0.5),
    VerticalFlip(p=0.5),
]

In [5]:
train.head()

,ImageId_ClassId,EncodedPixels
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0002cc93b.jpg_2,NaN
2,0002cc93b.jpg_3,NaN
3,0002cc93b.jpg_4,NaN
4,00031f466.jpg_1,NaN


In [6]:
df = pd.read_csv('./severstal-steel-defect-detection/train.csv')
# https://www.kaggle.com/amanooo/defect-detection-starter-u-net
df['ImageId'], df['ClassId'] = zip(*df['ImageId_ClassId'].str.split('_'))
df['ClassId'] = df['ClassId'].astype(int)
df = df.pivot(index='ImageId',columns='ClassId',values='EncodedPixels')
df['defects'] = df.count(axis=1)

In [7]:
df.head()

ClassId,1,2,3,4,defects
ImageId,,,,,
0002cc93b.jpg,29102 12 29346 24 29602 24 29858 24 30114 24 3...,NaN,NaN,NaN,1
00031f466.jpg,NaN,NaN,NaN,NaN,0
000418bfc.jpg,NaN,NaN,NaN,NaN,0
000789191.jpg,NaN,NaN,NaN,NaN,0
0007a71bf.jpg,NaN,NaN,18661 28 18863 82 19091 110 19347 110 19603 11...,NaN,1


In [8]:
#https://www.kaggle.com/paulorzp/rle-functions-run-lenght-encode-decode
def mask2rle(img):
    '''
    img: numpy array, 1 -> mask, 0 -> background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def make_mask(row_id, df):
    '''Given a row index, return image_id and mask (256, 1600, 4) from the dataframe `df`'''
    fname = df.iloc[row_id].name
    labels = df.iloc[row_id][:4]
    masks = np.zeros((256, 1600, 4), dtype=np.float32) # float32 is V.Imp
    # 4:class 1～4 (ch:0～3)

    for idx, label in enumerate(labels.values):
        if label is not np.nan:
            label = label.split(" ")
            positions = map(int, label[0::2])
            length = map(int, label[1::2])
            mask = np.zeros(256 * 1600, dtype=np.uint8)
            for pos, le in zip(positions, length):
                mask[pos:(pos + le)] = 1
            masks[:, :, idx] = mask.reshape(256, 1600, order='F')
    return fname, masks

In [9]:
def get_transforms(transforms, mean, std):
    list_transforms = []
    
    if transforms != None:
        list_transforms.extend(transforms)
        
    list_transforms.extend(
        [
            Resize(128,800),
            Normalize(mean=mean, std=std, p=1),
            ToTensor(),
        ]
    )
    list_trfms = Compose(list_transforms)
    return list_trfms
    

class SteelDataset(Dataset):
    def __init__(self, df, data_folder, transforms, mean, std):
        self.df = df
        self.root = data_folder
        self.mean = mean
        self.std = std
        self.transforms = get_transforms(transforms, mean, std)
        self.fnames = self.df.index.tolist()

    def __getitem__(self, idx):
        image_id, mask = make_mask(idx, self.df)
        image_path = os.path.join(self.root, "train_images",  image_id)
        img = np.asarray(Image.open(image_path))
        augmented = self.transforms(image=img, mask=mask)
        img = augmented['image']
        mask = augmented['mask'] # 1x256x1600x4
        mask = mask[0].permute(2, 0, 1) # 1x4x256x1600
        
        return img, mask

    def __len__(self):
        return len(self.fnames)

In [10]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [11]:
# define hyperparameters
test_split = 0.2
batch_size = 16
epochs = 30
learning_rate = 0.001
num_workers = 0
threshold = 0.5

In [12]:
# create dataset and data loaders (for FISH)
train_ds = SteelDataset(df, data_folder='./severstal-steel-defect-detection/', 
                        transforms=train_transforms, mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
test_ds = SteelDataset(df, data_folder='./severstal-steel-defect-detection/', 
                        transforms=None, mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))

dataset_size = len(train_ds)

indices = list(range(dataset_size))
split = int(np.floor(test_split * dataset_size))
np.random.seed(42)
np.random.shuffle(indices)
train_indices, test_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

trainloader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, sampler=train_sampler, num_workers=num_workers)
testloader = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, sampler=test_sampler, num_workers=num_workers)

In [13]:
import math
import torch
from torch.optim.optimizer import Optimizer, required

class RAdam(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        self.buffer = [[None, None, None] for ind in range(10)]
        super(RAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                buffered = self.buffer[int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        step_size = group['lr'] * math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    else:
                        step_size = group['lr'] / (1 - beta1 ** state['step'])
                    buffered[2] = step_size

                if group['weight_decay'] != 0:
                    p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)

                # more conservative since it's an approximated value
                if N_sma >= 5:            
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size, exp_avg, denom)
                else:
                    p_data_fp32.add_(-step_size, exp_avg)

                p.data.copy_(p_data_fp32)

        return loss

In [14]:
#source: https://www.kaggle.com/wh1tezzz/correct-dice-metrics-for-this-competition

def dice_channel_torch(probability, truth, threshold):
    batch_size = truth.shape[0]
    channel_num = truth.shape[1]
    mean_dice_channel = 0.
    with torch.no_grad():
        for i in range(batch_size):
            for j in range(channel_num):
                channel_dice = dice_single_channel(probability[i, j,:,:], truth[i, j, :, :], threshold)
                mean_dice_channel += channel_dice/(batch_size * channel_num)
    return mean_dice_channel


def dice_single_channel(probability, truth, threshold, eps = 1E-9):
    p = (probability.view(-1) > threshold).float()
    t = (truth.view(-1) > 0.5).float()
    dice = (2.0 * (p * t).sum() + eps)/ (p.sum() + t.sum() + eps)
    return dice

In [39]:
def iou_score(output, target):
    smooth = 1e-5

    if torch.is_tensor(output):
        output = torch.sigmoid(output).data.cpu().numpy()
    if torch.is_tensor(target):
        target = target.data.cpu().numpy()
    output_ = output > 0.5
    target_ = target > 0.5
    intersection = (output_ & target_).sum()
    union = (output_ | target_).sum()

    return (intersection + smooth) / (union + smooth)

In [15]:
class BCESoftDiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True, w1=1, w2=1):
        super(BCESoftDiceLoss, self).__init__()
        self.bce_loss = nn.BCELoss(weight, size_average)
        self.w1 = w1
        self.w2 = w2
        
    def forward(self, logits, targets):
        # BCELoss2d
        probs        = torch.sigmoid(logits)
        probs_flat   = probs.view (-1)
        targets_flat = targets.view(-1)
        bce_loss = self.bce_loss(probs_flat, targets_flat)
        
        # SoftDiceLoss
        num = targets.size(0)
        probs = torch.sigmoid(logits)
        m1  = probs.view(num,-1)
        m2  = targets.view(num,-1)
        intersection = (m1 * m2)

        score = 2. * (intersection.sum(1)+1) / (m1.sum(1) + m2.sum(1)+1)
        soft_dice_loss = 1- score.sum()/num
        
        return self.w1 * bce_loss + self.w2 * soft_dice_loss

In [16]:
encoder_name = 'efficientnet-b0'

In [17]:
model = smp.FPN(encoder_name, encoder_weights="imagenet", classes=4, activation=None)
model = model.to(device)
model

FPN(
  (encoder): EfficientNetEncoder(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          32, 8, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          8, 32, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_p

In [18]:
optimizer = RAdam(model.parameters(), lr=learning_rate)
criterion = BCESoftDiceLoss()

D:\Anaconda3\envs\Steel\lib\site-packages\torch\nn\_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [19]:
train_stats = pd.DataFrame(columns = ['Epoch', 
                                      'Time per epoch', 
                                      'Avg time per step', 
                                      'Train loss',
                                      'DICE Train',
                                      'Test loss',
                                     'DICE Test']) 

In [20]:
def train_model(model, device, trainloader, testloader, epochs, criterion, optimizer, train_stats):
    #train the model
    model = model.to(device)
    
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.15, patience=2)

    steps = 0
    running_loss = 0
    dice_train = 0
    for epoch in range(epochs):

        since = time.time()

        train_accuracy = 0
        dice_train = 0
        iou_train = 0
        
        for inputs, labels in tqdm_notebook(trainloader):
            steps += 1
            # Move input and label tensors to the default device
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            logps = model.forward(inputs)
            
            loss = criterion(logps, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            
            #dice_train += dice_channel_torch(logps, labels, threshold)
            
            #iou_train += iou_score(logps, labels)

        time_elapsed = time.time() - since

        test_loss = 0
        dice_test = 0
        iou_test = 0
        model.eval()
        with torch.no_grad():
            for inputs, labels in testloader:
                inputs, labels = inputs.to(device), labels.to(device)
                logps = model.forward(inputs)
                batch_loss = criterion(logps, labels)

                test_loss += batch_loss.item()

                # Calculate DICE
                dice_test += dice_channel_torch(logps, labels, threshold)
                
                #iou_test += iou_score(logps, labels)

        print(f"Epoch {epoch+1}/{epochs}.. "
              f"Time per epoch: {time_elapsed:.4f}.. "
              f"Average time per step: {time_elapsed/len(trainloader):.4f}.. "
              f"Train loss: {running_loss/len(trainloader):.4f}.. "
              f"Test loss: {test_loss/len(testloader):.4f}.. "
              f"DICE Train: {dice_train/len(trainloader):.4f}.. "
              f"DICE Test: {dice_test/len(testloader):.4f}.. "
              #f"IOU Train: {iou_train/len(trainloader):.4f}.. "
              #f"IOU Test: {iou_test/len(testloader):.4f}.. "
             )

        train_stats = train_stats.append({'Epoch': epoch + 1, 
                                          'Time per epoch':time_elapsed, 
                                          'Avg time per step': time_elapsed/len(trainloader), 
                                          'Train loss' : running_loss/len(trainloader),
                                          'Test loss' : test_loss/len(testloader),
                                          'DICE Train' : dice_train/len(trainloader),
                                          'DICE Test' : dice_test/len(testloader),
                                         }, 
                                         ignore_index=True)
                
        scheduler.step(test_loss/len(testloader))
        running_loss = 0
        model.train()
        
    return model, train_stats

In [ ]:
# train the model
torch.set_default_tensor_type("torch.cuda.FloatTensor")
model, train_stats = train_model(model, device, trainloader, testloader, epochs, criterion, optimizer, train_stats)

In [21]:
filename = 'efficientnet_b0_' + str(epochs) + '.pth'

checkpoint = {'state_dict': model.state_dict()}

torch.save(checkpoint, filename)

In [ ]:
# save training stats
train_stats.to_csv('efficientnet_b0_stats.csv', index=False)

In [ ]:
plt.plot(train_stats['Test loss'], label="test")
plt.plot(train_stats['Train loss'], label="train")
plt.ylim(0, 1.4)
plt.xlabel("epoch")
plt.ylabel("loss")
plt.legend(loc='best')

plt.show()

In [ ]:
plt.plot(train_stats['DICE Train'], label="test")
plt.plot(train_stats['DICE Test'], label="train")
plt.ylim(0, 1)
plt.xlabel("epoch")
plt.ylabel("dice")
plt.legend(loc='best')

plt.show()

In [40]:
test_loss = 0
dice_test = 0
iou = 0
model.eval()
with torch.no_grad():
    for inputs, labels in tqdm_notebook(testloader):
        inputs, labels = inputs.to(device), labels.to(device)
        logps = model.forward(inputs)
        batch_loss = criterion(logps, labels)

        test_loss += batch_loss.item()

        # Calculate DICE
        dice_test += dice_channel_torch(logps, labels, threshold)
        
        iou += iou_score(logps, labels)
        
test_loss = test_loss / len(testloader)
dice_test = dice_test / len(testloader)
iou = iou / len(testloader)

In [41]:
print('Test loss: {}'.format(test_loss))
print('DICE test: {}'.format(dice_test))
print('IOU test: {}'.format(iou))

Test loss: -0.16460924175909802
DICE test: 0.9147161245346069
IOU test: 0.4975083524678169


In [24]:
checkpoint = torch.load(filename)

In [25]:
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [26]:
model

FPN(
  (encoder): EfficientNetEncoder(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          32, 8, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          8, 32, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_p

In [27]:
#https://www.kaggle.com/paulorzp/rle-functions-run-lenght-encode-decode
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [28]:
class TestDataset(Dataset):
    '''Dataset for test prediction'''
    def __init__(self, root, df, mean, std):
        self.root = root
        df['ImageId'] = df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
        self.fnames = df['ImageId'].unique().tolist()
        self.num_samples = len(self.fnames)
        self.transform = Compose(
            [
                Normalize(mean=mean, std=std, p=1),
                ToTensor(),
            ]
        )

    def __getitem__(self, idx):
        fname = self.fnames[idx]
        path = os.path.join(self.root, fname)
        image = cv2.imread(path)
        images = self.transform(image=image)["image"]
        return fname, images

    def __len__(self):
        return self.num_samples

In [29]:
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
sample_submission_path = './severstal-steel-defect-detection/sample_submission.csv'
test_data_folder = TEST_PATH
df = pd.read_csv(sample_submission_path)
testset = DataLoader(
    TestDataset(test_data_folder, df, mean, std),
    batch_size=2,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

In [30]:
torch.cuda.empty_cache()

In [31]:
model = model.eval()

In [33]:
def post_process(probability, threshold, min_size):
    '''Post processing of each predicted mask, components with lesser number of pixels
    than `min_size` are ignored'''
    mask = cv2.threshold(probability, threshold, 1, cv2.THRESH_BINARY)[1]
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
    predictions = np.zeros((256, 1600), np.float32)
    num = 0
    for c in range(1, num_component):
        p = (component == c)
        if p.sum() > min_size:
            predictions[p] = 1
            num += 1
    return predictions, num

In [35]:
best_threshold = threshold
min_size = 3500

# start prediction
predictions = []
for i, batch in enumerate(tqdm_notebook(testset)):
    fnames, images = batch
    batch_preds = torch.sigmoid(model(images.to(device)))
    batch_preds = batch_preds.detach().cpu().numpy()
    for fname, preds in zip(fnames, batch_preds):
        for cls, pred in enumerate(preds):
            pred, num = post_process(pred, best_threshold, min_size)
            rle = mask2rle(pred)
            name = fname + f"_{cls+1}"
            predictions.append([name, rle])

# save predictions to submission.csv
df = pd.DataFrame(predictions, columns=['ImageId_ClassId', 'EncodedPixels'])
df.to_csv("submission.csv", index=False)

In [37]:
df.head()

,ImageId_ClassId,EncodedPixels
0,004f40c73.jpg_1,
1,004f40c73.jpg_2,
2,004f40c73.jpg_3,
3,004f40c73.jpg_4,
4,006f39c41.jpg_1,


In [38]:
df.EncodedPixels.count()

7204